In [74]:
#Region and Geocoding (Written by Chantelle)
import googlemaps
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

#geopy settings
geolocator = Nominatim(user_agent="DAO2702")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=0.1)

#postal codes by region
central = ['01' , '02' , '03' , '04' , '05' , '06' , '07' , '08' , '09' , '10' , 
                '14' , '15' , '16' , '17' , '18' , '19' , '20' , '21' , '22' , '23' , 
                '24' , '25' , '26' , '27' , '28' , '29' , '30' , '31' , '32' , '33' , 
                '34' , '35' , '36' , '37' , '38' , '39' , '40' , '41' , '58' , '59' , 
                '77' , '78']
west = ['11' , '12' , '13' , '60' , '61' , '62' , '63' , '64' , '65' , '66' , '67' , '68']
east = ['42' , '43' , '44' , '45' , '46' , '47' , '48' , '49' , '50' , '81' , '51' , '52']
north = ['69' , '70' , '71' , '72' , '73' , '75' , '76']
northeast = ['53' , '54' , '55' , '82' , '56' , '57' , '79' , '80']

#import address data
df = pd.read_csv("sampleaddresses.csv")
df.head(6)

,school_name,address,postal_code,Full Address
0,ADMIRALTY PRIMARY SCHOOL,11 WOODLANDS CIRCLE,738907,11 WOODLANDS CIRCLE 738907
1,ADMIRALTY SECONDARY SCHOOL,31 WOODLANDS CRESCENT,737916,31 WOODLANDS CRESCENT 737916
2,AHMAD IBRAHIM PRIMARY SCHOOL,10 YISHUN STREET 11,768643,10 YISHUN STREET 11 768643
3,AHMAD IBRAHIM SECONDARY SCHOOL,751 YISHUN AVENUE 7,768928,751 YISHUN AVENUE 7 768928
4,AI TONG SCHOOL,100 Bright Hill Drive,579646,100 Bright Hill Drive 579646
5,ALEXANDRA PRIMARY SCHOOL,2A Prince Charles Crescent,159016,2A Prince Charles Crescent 159016


In [75]:
#to convert postal code to sector
def first2(string):
    return string[:2]

In [76]:
#sorting sector codes by region
def region(postal):
    code = first2(postal)
    if code in central:
        return 'central'
    elif code in west:
        return 'west'
    elif code in east:
        return 'east'
    elif code in north:
        return 'north'
    elif code in northeast:
        return 'northeast'
    else:
        return 0

In [77]:
#create series for region
df['postal_code'] = df['postal_code'].apply(lambda x: str(x).zfill(6))
df['region'] = df['postal_code'].apply(region)

df.head(6)

,school_name,address,postal_code,Full Address,region
0,ADMIRALTY PRIMARY SCHOOL,11 WOODLANDS CIRCLE,738907,11 WOODLANDS CIRCLE 738907,north
1,ADMIRALTY SECONDARY SCHOOL,31 WOODLANDS CRESCENT,737916,31 WOODLANDS CRESCENT 737916,north
2,AHMAD IBRAHIM PRIMARY SCHOOL,10 YISHUN STREET 11,768643,10 YISHUN STREET 11 768643,north
3,AHMAD IBRAHIM SECONDARY SCHOOL,751 YISHUN AVENUE 7,768928,751 YISHUN AVENUE 7 768928,north
4,AI TONG SCHOOL,100 Bright Hill Drive,579646,100 Bright Hill Drive 579646,northeast
5,ALEXANDRA PRIMARY SCHOOL,2A Prince Charles Crescent,159016,2A Prince Charles Crescent 159016,central


In [80]:
#create series for lat and long data
df['location'] = df['Full Address'].apply(geocode)
df['point'] = df['location'].apply(lambda loc: tuple(loc.point) if loc else None)

df[['lat', 'lon', 'alt']] = pd.DataFrame(df['point'].tolist(), index=df.index)
df = df.drop('alt', axis=1)

df.head(6)

,school_name,address,postal_code,Full Address,region,location,point,lat,lon
0,ADMIRALTY PRIMARY SCHOOL,11 WOODLANDS CIRCLE,738907,11 WOODLANDS CIRCLE 738907,north,"(Admiralty Primary School, 11, Woodlands Circl...","(1.4429414, 103.80034506744822, 0.0)",1.442941,103.800345
1,ADMIRALTY SECONDARY SCHOOL,31 WOODLANDS CRESCENT,737916,31 WOODLANDS CRESCENT 737916,north,"(Admiralty Secondary School, 31, Woodlands Cre...","(1.4459118499999999, 103.80290805413242, 0.0)",1.445912,103.802908
2,AHMAD IBRAHIM PRIMARY SCHOOL,10 YISHUN STREET 11,768643,10 YISHUN STREET 11 768643,north,"(Ahmad Ibrahim Primary School, 10, Yishun Stre...","(1.4338487, 103.83270977683569, 0.0)",1.433849,103.832710
3,AHMAD IBRAHIM SECONDARY SCHOOL,751 YISHUN AVENUE 7,768928,751 YISHUN AVENUE 7 768928,north,"(Yishun Avenue 7, Yishun, Northwest, 768928, S...","(1.4361068, 103.8279176, 0.0)",1.436107,103.827918
4,AI TONG SCHOOL,100 Bright Hill Drive,579646,100 Bright Hill Drive 579646,northeast,"(Ai Tong School, 100, Bright Hill Drive, Bisha...","(1.3607125500000001, 103.83301753795786, 0.0)",1.360713,103.833018
5,ALEXANDRA PRIMARY SCHOOL,2A Prince Charles Crescent,159016,2A Prince Charles Crescent 159016,central,"(Alexandra Primary School, 2A, Prince Charles ...","(1.2912837000000001, 103.82406594082047, 0.0)",1.291284,103.824066


In [83]:
#convert data to csv file
df.to_csv('sampleresult.csv', index=False)